In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb


/home/natalie/anaconda3/envs/insight/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/natalie/anaconda3/envs/insight/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


We will be looking at school specific data from across the United States. Our goal is to see if any schools resemble eachother. Often schools are described as "good", but if you are not familiar with the area you have little idea what that definition of "good" means. We want to explore the school graduation rates, diversity, expulsion rates, and teacher ratios to get an idea of what a "good" school is.

If we are also able to cluster like schools together, you may be able to see how your current school (something you are familiar with) compares to other schools you are looking to move into.

## Data sources:

2015-2016 graduation rates: https://www2.ed.gov/about/inits/ed/edfacts/data-files/index.html#acgr

2015-2016 school location, characteristics, staff, and lunch programs: https://nces.ed.gov/ccd/pubschuniv.asp

2015-2016 school achievement: https://www2.ed.gov/about/inits/ed/edfacts/data-files/index.html#acgr
    

In [3]:
#load data
path='/home/natalie/Documents/SchoolAPI/Data_Sources/'
grad_2015=pd.read_csv(path+'graduation_rates/2015_school_graduation.csv', parse_dates=True)


In [4]:
#school specific data
school_lunch=pd.read_csv(path+'CCD_comoncoredata_nonfiscal/school_lunches_ccd_sch_033_1516.csv',  parse_dates=True)
school_char=pd.read_csv(path+'CCD_comoncoredata_nonfiscal/school_characteristics_ccd_sch_129_1516_w_2a_011717.csv', parse_dates=True)
school_dir =pd.read_csv(path+'CCD_comoncoredata_nonfiscal/school_directory_ccd_sch_029_1516_w_2a_011717.csv',  parse_dates=True)
school_member=pd.read_csv(path+ 'CCD_comoncoredata_nonfiscal/schools_membership_ccd_sch_052_1516_w_2a_011717.csv' ,  parse_dates=True)


/home/natalie/anaconda3/envs/insight/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6,14,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/natalie/anaconda3/envs/insight/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
#School achievement results for math and reading

math=pd.read_csv(path+'Subject_assesments/school_math-achievement-sch-sy2015-16.csv', parse_dates=True, low_memory=False)
reading= pd.read_csv(path+'Subject_assesments/rla-achievement-sch-sy2015-16.csv', parse_dates=True, low_memory=False)

# percent is the % above the state's "proficient" level
#colums starting with F or M denote breakdowns by gender


In [20]:
math=pd.read_csv(path+'Subject_assesments/school_math-achievement-sch-sy2015-16.csv', parse_dates=True, low_memory=False)

In [ ]:
school_char[school_char['NCESSCH'] == 450231000049] #this matches the greenville download for suspensions from civil rights data

In [6]:
#greenville suspensions
suspensions=pd.read_csv(path+'civil_rights/suspensions_greenville_2015.csv')

In [7]:
suspensions.head()
#n = individual counts of students
#otherwise in percentage
#times white students is a multiplier of how much more likely they are to be suspended than a white student

,District ID,District,State,School_ID,School Name,AME_n_susp,ASI_n_susp,HI PAC_n_susp,BLA_n_susp,HIS_n_susp,...,HIS_OSS_times_white,TWO_OSS_times_white,AME_n_enrollment,ASI_n_enrollment,HI_PAC_n_enrollment,BLA_n_enrollment,HIS_n_enrollment,WHI_n_enrollment,TWO_n_enrollment,TOT_n_enrollment
0,4502310,Greenville 01,SC,450231000049,Heritage Elementary,0,0,0,2,0,...,0.00,0.00,0,4,0,61,16,565,25,671
1,4502310,Greenville 01,SC,450231000114,Oakview Elementary,0,0,0,6,0,...,0.00,0.00,10,79,0,91,37,697,46,960
2,4502310,Greenville 01,SC,450231000449,Chandler Creek Elementary,0,2,0,12,6,...,1.84,0.00,19,4,0,214,190,526,64,1017
3,4502310,Greenville 01,SC,450231000455,Mauldin Middle,0,2,0,37,4,...,0.77,3.62,4,37,2,214,82,631,55,1025
4,4502310,Greenville 01,SC,450231000459,Riverside Middle,0,0,0,20,7,...,1.36,3.53,2,76,2,121,109,805,49,1164


In [8]:
#G12OFFERED = YES then this is a high school
#G1OFFERED = YES and G12OFFERED=NO then this is an elementary school
#first replace yes and no with 1 and 0, respectively

school_dir=school_dir.replace(['yes', 'Yes'], 1, regex=True)
#school_dir1=school_dir.replace('Yes', 1)
school_dir=school_dir.replace('No', 0, regex=True)

school_dir.head()

,SURVYEAR,FIPST,STABR,STATENAME,SEANAME,LEAID,ST_LEAID,LEA_NAME,SCHID,ST_SCHID,...,G13OFFERED,AEOFFERED,UGOFFERED,NOGRADES,CHARTAUTH1,CHARTAUTHN1,CHARTAUTH2,CHARTAUTHN2,IGOFFERED,WEBSITE
0,2015-2016,1,AL,ALABAMA,Alabama Department Of Education,100002,210,Alabama Youth Services,1667,210-0050,...,0,0,0,0,NOT APPLICABLE,NOT APPLICABLE,NOT APPLICABLE,NOT APPLICABLE,R,www.dys.alabama.gov
1,2015-2016,1,AL,ALABAMA,Alabama Department Of Education,100002,210,Alabama Youth Services,1876,210-0001,...,0,0,0,1,NOT APPLICABLE,NOT APPLICABLE,NOT APPLICABLE,NOT APPLICABLE,R,NaN
2,2015-2016,1,AL,ALABAMA,Alabama Department Of Education,100002,210,Alabama Youth Services,1706,210-0040,...,0,0,0,0,NOT APPLICABLE,NOT APPLICABLE,NOT APPLICABLE,NOT APPLICABLE,R,www.dys.alabama.gov
3,2015-2016,1,AL,ALABAMA,Alabama Department Of Education,100002,210,Alabama Youth Services,1670,210-0060,...,0,0,0,0,NOT APPLICABLE,NOT APPLICABLE,NOT APPLICABLE,NOT APPLICABLE,R,www.dys.alabama.gov
4,2015-2016,1,AL,ALABAMA,Alabama Department Of Education,100002,210,Alabama Youth Services,277,210-0020,...,0,0,0,0,NOT APPLICABLE,NOT APPLICABLE,NOT APPLICABLE,NOT APPLICABLE,R,www.dys.alabama.gov


In [9]:
school_dir['type_school']=np.nan

#df['elderly'] = np.where(df['age']>=50, 'yes', 'no')
      
school_dir['type_school'] = np.where((school_dir['G1OFFERED']==1) & (school_dir['G12OFFERED']==0) , 'elementary',
                np.where((school_dir['G1OFFERED']==0) & (school_dir['G12OFFERED']==1) , 'high_school', 
                    np.where((school_dir['G1OFFERED']==0) & (school_dir['G12OFFERED']==0)& (school_dir['G8OFFERED']==1) , 'middle',  np.nan)))


In [10]:
school_dir[['G1OFFERED', 'G8OFFERED', 'G12OFFERED',  'type_school']].head(10)

,G1OFFERED,G8OFFERED,G12OFFERED,type_school
0,0,1,1,high_school
1,0,0,0,nan
2,0,1,1,high_school
3,0,1,1,high_school
4,0,1,1,high_school
5,0,1,1,high_school
6,0,0,0,nan
7,0,0,0,nan
8,0,1,0,middle
9,0,0,0,nan


In [ ]:
grad_2015.head()

In [15]:
reading.head()

,STNAM,FIPST,LEAID,LEANM,NCESSCH,SCHNAM,DATE_CUR,ALL_RLA00NUMVALID_1516,ALL_RLA00PCTPROF_1516,MAM_RLA00NUMVALID_1516,...,MIG_RLA05NUMVALID_1516,MIG_RLA05PCTPROF_1516,MIG_RLA06NUMVALID_1516,MIG_RLA06PCTPROF_1516,MIG_RLA07NUMVALID_1516,MIG_RLA07PCTPROF_1516,MIG_RLA08NUMVALID_1516,MIG_RLA08PCTPROF_1516,MIG_RLAHSNUMVALID_1516,MIG_RLAHSPCTPROF_1516
0,ALABAMA,1,100005,Albertville City,10000500870,Ala Avenue Middle Sch,14APR17,687,33,2,...,NaN,NaN,NaN,NaN,17,LE20,17,LE20,NaN,NaN
1,ALABAMA,1,100005,Albertville City,10000500871,Albertville High Sch,14APR17,307,31,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,LT50
2,ALABAMA,1,100005,Albertville City,10000500879,Evans Elem Sch,14APR17,759,30,1,...,23,LE20,14,LT50,NaN,NaN,NaN,NaN,NaN,NaN
3,ALABAMA,1,100005,Albertville City,10000500889,Albertville Elem Sch,14APR17,839,36,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ALABAMA,1,100006,Marshall County,10000600193,Kate Duncan Smith DAR Middle,14APR17,442,42,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Data CLEANING

#### 1. Let's first filter the datasets to only include the variables we want.
These variables are:

grad_2015:

math/reading:
STNAM = state name

LEANM = education agency name

NCESSCH = unique school ID

SCHNAM = school name

ALL_MTH00NUMVALID_1516  = number of students aggregated across all grades who took test
ALL_MTH00PCTPROF_1516= percentage of all students who are proficient in math across all grades

F_MTH00NUMVALID_1516 = females who took math exam across all grades
F_MTH00PCTPROF_1516 = females proficient in math across all grades

M_MTH00NUMVALID_1516 = males who took math exam across all grades
M_MTH00PCTPROF_1516 = males proficient in math across all grades

MBL_MTHHSNUMVALID_1516 = number of black students in high school who took math examns
MBL_MTHHSPCTPROF_1516 = percent of black students in high school who are proficient in math

MWH_MTHHSNUMVALID_1516 = Number of white students in high school who took math exams
MWH_MTHHSPCTPROF_1516 = percent of white students in high school who are proficient in math

same thing for reading, but MTH is replaced with RLA

In [17]:
#keep only variables listed above

math_keep=['STNAM', 'LEANM', 'NCESSCH', 'SCHNAM', 'ALL_MTH00NUMVALID_1516', 'ALL_MTH00PCTPROF_1516', 'F_MTH00NUMVALID_1516', 'F_MTH00PCTPROF_1516', 'M_MTH00NUMVALID_1516', 'M_MTH00PCTPROF_1516', 'MBL_MTHHSNUMVALID_1516', 'MBL_MTHHSPCTPROF_1516' , 'MWH_MTHHSNUMVALID_1516', 'MWH_MTHHSPCTPROF_1516']
math=math[math_keep]


reading_keep=['STNAM', 'LEANM', 'NCESSCH', 'SCHNAM', 'ALL_RLA00NUMVALID_1516', 'ALL_RLA00PCTPROF_1516', 'F_RLA00NUMVALID_1516', 'F_RLA00PCTPROF_1516', 'M_RLA00NUMVALID_1516', 'M_RLA00PCTPROF_1516', 'MBL_RLAHSNUMVALID_1516', 'MBL_RLAHSPCTPROF_1516' , 'MWH_RLAHSNUMVALID_1516', 'MWH_RLAHSPCTPROF_1516']
reading=reading[reading_keep]

In [19]:
math.head(11)

,STNAM,LEANM,NCESSCH,SCHNAM,ALL_MTH00NUMVALID_1516,ALL_MTH00PCTPROF_1516,F_MTH00NUMVALID_1516,F_MTH00PCTPROF_1516,M_MTH00NUMVALID_1516,M_MTH00PCTPROF_1516,MBL_MTHHSNUMVALID_1516,MBL_MTHHSPCTPROF_1516,MWH_MTHHSNUMVALID_1516,MWH_MTHHSPCTPROF_1516
0,ALABAMA,Albertville City,10000500870,Ala Avenue Middle Sch,707,36,347,39,360,34,NaN,NaN,NaN,NaN
1,ALABAMA,Albertville City,10000500871,Albertville High Sch,307,15,161,15-19,146,10-14,9,LT50,177,20-24
2,ALABAMA,Albertville City,10000500879,Evans Elem Sch,772,43,389,46,383,40,NaN,NaN,NaN,NaN
3,ALABAMA,Albertville City,10000500889,Albertville Elem Sch,852,55,421,54,431,57,NaN,NaN,NaN,NaN
4,ALABAMA,Marshall County,10000600193,Kate Duncan Smith DAR Middle,443,36,206,30-34,237,35-39,NaN,NaN,NaN,NaN


#### 1. the rates of graduation or math/reading competence can be given in values '50-60', or 'GE50' = greater than 50%, or 'PS' which means the data was intentially hidden so you could not focus in on a very small group (e.g., 2 students are hispanic, so any data on these 2 students would wrongly expose their identity). To be conservative, we will select the lower range 50-60, we would select '50' to represent the graduation rate. For values with GE greater than or LE less than, we simply select the value stated, so GE80 becomes '80'.

In [ ]:
import re
grad_variables=list(grad_2015)

r=re.compile(".*RATE")
rate_variables = list(filter(r.match, grad_variables)) # Read Note

In [ ]:
replace_list=['GE', 'LE', 'LT']

for word in replace_list:
    reading=reading.replace(word, '', regex=True)
    math=math.replace(word, '', regex=True)
    grad_2015=grad_2015.replace(word, '', regex=True) #regex=TRUE checks for partial match, otherwise it would need to match it 100%

na_list=['.', 'PS']

for word in na_list:
    reading=reading.replace(word, np.nan, regex=True)
    math=math.replace(word, np.nan, regex=True)
    grad_2015=grad_2015.replace(word, np.nan, regex=True)

In [19]:
#removehe the ranges, instead select the lowest value in the range, if rate is 80-85, we only want to keep 80
#grad_2015['ALL_RATE_1516'] = grad_2015['ALL_RATE_1516'].str.split('-').str[0]

for variable in rate_variables:
    grad_2015[variable]= grad_2015[variable].str.split('-').str[0]

r=re.compile(".PROF")
prof_variables=list(filter(r.match, list(math)))

for variable in prof_variables:
    math[variable]= math[variable].str.split('-').str[0]

In [20]:
#same thing remove dashes in reading variables
r=re.compile(".PROF")
prof_variables=list(filter(r.match, list(reading)))

for variable in prof_variables:
    reading[variable]= reading[variable].str.split('-').str[0]


In [21]:
math.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89409 entries, 0 to 89408
Columns: 231 entries, STNAM to MIG_MTHHSPCTPROF_1516
dtypes: int64(4), object(227)
memory usage: 157.6+ MB


In [ ]:
grad_2015.info()

In [13]:
#Change all the Cohort variables (those with specific numbers of students to float instead of object)
r=re.compile(".*COHORT")
cohort_variables = list(filter(r.match, grad_variables))

for variable in rate_variables:
    grad_2015[variable]=grad_2015[variable].astype('float')
    
for variable in cohort_variables:
    grad_2015[variable]=grad_2015[variable].astype('float')

## The "All_Cohort" variable is all students in that school, then they are broken down into racial groups.

MAM = American Indian/Alaska Native students

MAS = Asian/Pacific Islander

MHI = Hispanic/Latino

MBL = Black

MWH = White

MTR = Two or more races

CWD = disabilities

ECD = Economically disadvanted

LEP = Limited English Proficiency

In [1]:
grad_2015.describe() #everything we would expect is numeric and can be summarized. These columns can also be used to creat a distance matrix between the different schools.

NameError: name 'grad_2015' is not defined

Clustering algorithims cannot handle missing data, what data is missing?

In [59]:
na_list=grad_2015.isna().sum()
na_list/len(grad_2015) # percentage
#Because many of these race groups are so small the PS identifier was used, and then converted to NA.

STNAM              0.0
FIPST              0.0
LEAID              0.0
LEANM              0.0
NCESSCH            0.0
SCHNAM             0.0
ALL_COHORT_1516    0.0
ALL_RATE_1516      0.0
MAM_COHORT_1516    0.0
MAM_RATE_1516      0.0
MAS_COHORT_1516    0.0
MAS_RATE_1516      0.0
MBL_COHORT_1516    0.0
MBL_RATE_1516      0.0
MHI_COHORT_1516    0.0
MHI_RATE_1516      0.0
MTR_COHORT_1516    0.0
MTR_RATE_1516      0.0
MWH_COHORT_1516    0.0
MWH_RATE_1516      0.0
CWD_COHORT_1516    0.0
CWD_RATE_1516      0.0
ECD_COHORT_1516    0.0
ECD_RATE_1516      0.0
LEP_COHORT_1516    0.0
LEP_RATE_1516      0.0
DATE_CUR           0.0
kmeans             0.0
dtype: float64

In [17]:
# Fill missing values with mean column values in those columns
grad_2015.fillna(grad_2015.mean(), inplace=True)

### Clustering on just the graduation data did not seem to help, let's see if pulling in other data will improve our model.



Merge all the data for individual schools together, using 'NCESSCH' as the key id variable.

In [ ]:
all_data=pd.merge(df_new, grad_2015, on='subject_id')